# Outline

In this notebook we will develop a Bayesian Network (BN) class and some of the key algorithms for BNs.

A BN combines a DAG and a collection of CPDs, and we have coded DAGs and CPDs (specifically, tabular CPDs) for you. We also provide some of the code for a BN class, and you will complete some of it.

This is a high-level outline of the notebook, you will find exercises in most sections.

1. We start with constructing DAGs
2. Next, we construct tabular CPDs
3. Then, we design a BN class combining DAGs and tabular CPDs, this class will support all three fundamental probability queries (joint, marginal and conditional).
4. We then look into reasoning patterns.
5. Finally, we test for independence in two ways, namely, using the joint distribution and using only the BN structure.

**Table of Exercises**

The exercises and the points they are worth are shown below:

1. Student: DAG [0.5]
2. Student: CPDs [0.5]
3. Student: BN [0.5]
4. Student: Joint Distribution [1]
5. Student: Reasoning Patterns [1.5]
6. Student: Independence [2]
7. Student: Trails [1.5]
8. Student: D-Separation [1]
9. Extended Student: BN Structure [0.5]
10. Extended Student: D-Separation [1]


**Use of AI tools**

In this course we expect _you_ and your team members to author your work.
AI tools are not to be used for drafts, nor code completion, nor revisions, nor as a 'study tool', nor as a source of feedback. If you do use AI, it should not contribute to the substance of what you present as your work.  

At the end of this notebook you will find a section on _Use of AI tools_. **Make sure to read and complete it**.
By submitting a version of this notebook for assessment, you agree with our terms.

# Setting Up

Take care of dependencies:

In [ ]:
# !pip install tabulate
# !pip install git+https://github.com/probabll/pgmini.git

In [ ]:
import pgmini
print(pgmini.__version__)

In [ ]:
from collections import defaultdict, deque, OrderedDict
import itertools
from tabulate import tabulate
import numpy as np

# DAGs

We use a few helpers from [pgmini](https://github.com/probabll/pgmini), you can check a demo notebook for these helpers in pgmini's repository. In particular, we are using `pgmini.m1` (for Module 1).

In [ ]:
from pgmini.m1 import DAG

Let's start with the BN _structure_, namely, a **directed acyclic graph** (DAG).
Using our _DAG_ class you can build DAGs very easily:


In [ ]:
DAG(nodes=['A', 'B', 'C', 'D'], edges=[('B', 'A'), ('C', 'A'), ('D', 'B')])

# Tabular CPDs

In [ ]:
from pgmini.m1 import OutcomeSpace, TabularCPD

Next, we need _conditional probability distributions_ (CPDs). A CPD assigns probability to the outcomes of a random variable (rv) in a given conditioning context.

Consider two binary rvs, namely $B$ and $D$. Given $D=d^1$, we have a cpd over the outcomes of $B|D=d^1$, which specifies for example that $P(B=b^1|D=d^1)=0.9$ and $P(B=b^0|D=d^1)=0.1$. Given $D=d^0$, we have a cpd over the outcomes of $B|D=d^0$, which specifies for example that $P(B=b^1|D=d^0)=0.5$ and $P(B=b^0|D=d^0)=0.5$.

There are many ways to represent cpds in a computer, our strategy for now is to use a **table**. That's okay because for now we are only interested in discrete rvs (with countably finite sample spaces) and not too many of them will interact at once.

A **tabular CPD** is essentially a table whose rows identify the _conditioning context_ and whose columns identify the _outcomes_ of rv whose distribution we are specifying. In the example above, the table will have rows for $b^0$ and $b^1$ and columns for $a^0$ and $a^1$.

The table below illustrates the basic datastructure.

| context  &nbsp;&nbsp;&nbsp;&nbsp;      |           $ b^0 $           |           $ b^1 $          |
| :------------- | :-------------------------: | :-------------------------: |
|   $ D=d^0 $  | $ P(B = b^0 \mid D = d^0) $ | $ P(B = b^1 \mid D = d^0) $ |
|   $ D=d^1 $  | $ P(B = b^0 \mid D = d^1) $ | $ P(B = b^1 \mid D = d^1) $ |

Internally we use a tensor, rather than a table, each axis of the tensor is associated with the outcome space of one of the rvs, in order: the parent rvs followed by the child rv (last axis).

In [ ]:
cpdC = TabularCPD(
    [], 'C',
    {'C': OutcomeSpace(['c0', 'c1'])},
    table=[0.2, 0.8],
)
print(cpdC)

In [ ]:
cpdD = TabularCPD(
    [], 'D',
    {'D': OutcomeSpace(['d0', 'd1'])},
    table=[0.6, 0.4],
)
print(cpdD)

In [ ]:
cpdB = TabularCPD(
    ['D'], 'B',
    {'B': OutcomeSpace(['b0', 'b1']),
     'D': OutcomeSpace(['d0', 'd1'])
    },
    table=[[0.5, 0.5], [0.1, 0.9]],
)
print(cpdB)

In [ ]:
cpdA = TabularCPD(
    ['B', 'C'], 'A',
    {'B': OutcomeSpace(['b0', 'b1']),
     'C': OutcomeSpace(['c0', 'c1']),
     'A': OutcomeSpace(['a1', 'a2', 'a3']),
    },
    table=[[[0.3 , 0.6, 0.1], [0.7 , 0.1, 0.2]],
           [[0.45, 0.5, 0.05], [0.1 , 0.88, 0.02]]],
)
print(cpdA)

We can retrieve a specific cell by querying any of these CPDs with an assignment of the rvs:

In [ ]:
cpdA.prob({'A': 'a3', 'B': 'b0', 'C': 'c1', 'D': 'd1'})  # irrelevant rvs are ignored

# The Student Example

We are ready to code the graph structure of the Student BN discussed in class (Figure 1).

**EXERCISE - Student: DAG.** Construct a DAG for the BN structure of the Student example (Figure 1 in class) and display it.

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Student: CPDs.** Construct _TabularCPD_ objects for all CPDs in the Student example discussed in class (use the same numerical values as we did in class; Figure 2) and display them.

In [ ]:
# **YOUR SOLUTION HERE**

# Bayesian Network

Then, our **Bayesian Network** (BN) data structure will store a DAG for the BN structure and a collection of the CPDs for the nodes.

In [ ]:
class BayesianNetwork:
    """
    A BN combines a DAG (we use an implementation from pgmini.m1)
     and a collection of CPDs (we use TabularCPD from pgmini.m1)
    """

    def __init__(self, nodes: list, edges: list):
        """
        nodes: a list of pairs, each element is a tuple (rv_name: str, cpd: TabularCPD)
        edges: a list of pairs, each element is a tuple (parent_name: str, child_name: str)

        Remark: some students may object that it is possible to figure out 'nodes' and 'edges'
         from a collection of CPDs, while that is true it can lead to confusion as a CPD implementation
         allows for quite some room for variation; here we opt for a clearer (even if somewhat redundant)
         constructor.
        """
        self.dag = DAG([rv for rv, _ in nodes], edges)
        self.cpds = dict(nodes)  # rv -> CPD

    def joint_probability(self, assignment: dict):
        """
        Compute and return the joint probability (float) of an assignment of the rvs.
            This assignment is regarded as 'complete' so long for any rv is assigned its ancestors are also assigned
            For example, in the BN structure A -> B -> C,
                the assignments (A=a1), (A=a1, B=b0) and (A=a1, B=b0, C=c1) are 'complete'
                but the assignments (B=b0), (C=c1), (B=b1, C=c1) and (A=a1, C=c1) are not, for in each of these cases
                some necessary ancestors of a given rv is missing.

            You do not need to test if the assignment is 'complete' in this sense, you
            can assume that no one will query incomplete assignments (and if they do, they will get exceptions from TabularCPD).

            If you ever have to query an assignment that is incomplete in the sense above,
            you need to think about the _kind_ of query you are making and find a better method in this class.

        assignment: a dict mapping each rv to an outcome in the rv's outcome space
        """
        raise NotImplementedError("Implementing this is an exercise")

    def marginal_probability(self, query_assignment=dict()):
        """
        Compute P(query_assignment) by enumerating over unassigned ancestors only.
            That is, P(query_assignment) = \sum_{unassigned_ancestors} P(query_assignment, unassigned_ancestors)
        """
        query = query_assignment.keys()
        # gather all ancestors
        ancestors = set()
        for rv in query:
            ancestors |= self.dag.ancestors[rv]
        # remove the query variables (in case the query contains A=a, D=d and A is one of the ancestors of D)
        # these are the variables we want to marginalise out
        unassigned_ancestors = tuple(v for v in ancestors if v not in query)

        prob = 0.0
        # these are the outcome spaces of the variables to be marginalised out
        outcome_spaces = [self.cpds[ancestor].enumerate_outcomes() for ancestor in unassigned_ancestors]
        # here we enumerate the assignments in the cartesian product of the outcome spaces
        for outcomes in OutcomeSpace.enumerate_joint_outcomes(*outcome_spaces):
            missing_assignment = dict(zip(unassigned_ancestors, outcomes))
            # combining the missing assignment and the query assignment
            # we have an assignment that misses no ancestor
            prob += self.joint_probability({**query_assignment, **missing_assignment})

        return prob

    def conditional_probability(self, query_assignment: dict, evidence_assignment: dict):
        """
        Compute P(query | evidence) = P(query, evidence) / P(evidence)
        """
        # we use the definition of conditional probability: P(B|A) = P(A,B)/P(B)
        # where P(A,B) and P(B) are marginals of a joint distribution P(A,B,C), for example.
        numerator = self.marginal_probability({**evidence_assignment, **query_assignment})
        denominator = self.marginal_probability(evidence_assignment)
        return numerator / denominator

    def enumerate_joint_outcomes(self, rvs):
        """
        Helper to enumerate joint outcomes (list of strings) in the cartersian product (cross-product) space of the rvs' outcome spaces
        """
        outcome_spaces = [self.cpds[rv].enumerate_outcomes() for rv in rvs]
        for joint_outcome in itertools.product(*outcome_spaces):
            yield joint_outcome

    def enumerate_assignments(self, rvs):
        """
        Helper to enumerate joint assignments (dict mapping rv name to outcome) in the cartersian product (cross-product) space of the rvs' outcome spaces
        """
        for joint_outcome in self.enumerate_joint_outcomes(rvs):
            yield dict(zip(rvs, joint_outcome))

**EXERCISE - Student: BN.** Construct the a _BayesianNetwork_ object for the Student example (i.e., the DAG from Figure 1 and CPDs from Figure 2) and display its structure.

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Student: Joint Distribution.** Complete the `joint_probability` method of the `BayesianNetwork` class, then for the Student BN, display the complete joint distribution in a table and verify that it adds up to 1 over its joint outcome space.

Here's how you should format the table (you can use `tabulate` or your preferred helper for visualising tables):
* one joint outcome per row
* columns for each rv and a final column for the probability
* for probability, use 5 decimals of precision
* for the order of the joint outcomes, use `('D', 'I', 'G', 'S', 'L')`

This should reconstruct Table 1 from the class slides.

In [ ]:
# **YOUR SOLUTION HERE**

# Reasoning Patterns

We can use probability calculus to appreciate the effect of observing a variable may or may not have on our beliefs about other variables. The three broad categories of reasoning patterns due to observation are causal, evidential and intercausal.  

**EXERCISE - Student: Reasoning Patterns.** Make the relevant queries to the joint distribution and demonstrate examples of the following:

* 3 examples of _causal reasoning_, each on a different rv;
* 3 examples of _evidential reasoning_, each on a different rv;
* 2 examples of _intercausal reasoning_, where you vary the rv that's activating the v-structure

For each example, you must display the result of any probability query you use in your rationale for the example, and you must provide a brief explanation of why the example demonstrates causal, evidential or intercausal reasoning.

Tip: the strategy here is to compare the probability of the outcome of some rv before and after observing the outcome of some other rv, this demonstrates an effect; why this effect is causal, evidential or intercausal is for you to explain.

Make sure your solution is organised, your TA cannot grade it if they cannot understand it.

In [ ]:
# **YOUR SOLUTION HERE**

# Independence

When we have a complete representation of a joint distribution, we can reason about (conditional) independence. Here, you will do just that.

**EXERCISE - Student: Independence.** Test independence via probability calculus (that is, make the necessary joint/conditional/marginal queries to the BN, as to ascertain some independence statement) for the following statements:

1. $L \perp S$
2. $L \perp S \mid I$
4. $D \perp L$
5. $D \perp L \mid G$

You can implement a function that does the necessary computations and returns True or False, or you can implement a function that computes and prints the necessary probability tables for you to decide by inspection of results (this 'inspection' method is how we did it in class). If you opt for the 'inspection' strategy, you need to explain how you draw your conclusion, otherwise the TA cannot know your rationale for the answer. Also note that when we say _inspection_ we really mean it, one needs to be able to look at what you are referring to and see it, without having to do any further calculations.

In [ ]:
# **YOUR SOLUTION HERE**

# D-Separation

D-Separation is a tool for ascertaining (conditional) independence statements based solely on the BN structure (that is, knowing the DAG is enough, we do not need a complete distribution with its underlying local probability models).

D-separation depends on the concept of a trail, which may be active (enable influence) or inactive (block influence), as discussed in class. When we make a d-sep claim we are making a claim that holds for _all_ trials between the relevant nodes.

In general, in a DAG, enumerating all trials is an exponential-time algorithm, so implementing d-spearation requires careful algorithms.
A polynomial-time algorithm for D-separation is possible and not too complex, but for didactic purposes, here we will be _enumerating_ trails exhaustively and one by one (for this notebook this is okay because we only have small BNs).

You will first implement the test to determine whether a trail is active or not. After that, you will implement d-separation.

**EXERCISE - Student: Active Trails.**

1. Implement a helper function to decide whether a trail is active in a DAG.
2. Then use `make_all_trails_table` below to print the trails between all pairs of nodes in the Student BN and the trail's status (active or not) before and after observing an evidence set. Print a table with `evidence={'G'}` and one with `evidence={'L'}`.

In [ ]:
def make_all_trails_table(dag: DAG, evidence=set()):
    """
    Make a table for visualiasation to show:
        for all pairs of nodes in the dag,
            all trails and whether they are active before/after observing evidence
    """
    trail_table = []
    for x, y in itertools.combinations(dag.topo, 2):
        for trail in dag.enumerate_trails(x, y):
            trail_table.append([x, y, " -- ".join(trail), is_trail_active(dag, trail), is_trail_active(dag, trail, evidence)])
    return tabulate(trail_table, headers=['From', 'To', 'Trail', 'Active without evidence', f"Active given {', '.join(evidence)} "])

In [ ]:
def is_trail_active(dag: DAG, trail: list, evidence=set()):
    """
    Return True if a given trail (list of nodes) is active given evidence.
    """
    # **YOUR SOLUTION HERE***
    raise NotImplementedError("Implementing this is an exercise!")

In [ ]:
# **YOUR SOLUTION HERE***

**EXERCISE - Student: D-Separation.** Implement d-seperation by enumeration of trails. Test it for the following 4 statements:

1. $L \perp S$
2. $L \perp S \mid I$
4. $D \perp L$
5. $D \perp L \mid G$

The result should be the same as when you tested independence via probability calculus.

In [ ]:
def dsep(bn_structure: DAG, X: set(), Y: set, Z=set()):
    """
    Return True if d-sep(X; Y|Z) or False otherwise.
    """
    # **YOUR SOLUTION HERE**
    raise NotImplementedError("Implementing this is an exercise")

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Extended Student: BN Structure.** Construct the BN Structure for the _Extended_ Student Example discussed in class (Figure 3) and display it.

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Extended Student: D-Separation.** Use your code to test the following d-sep statements against the BN structure of the Extended Student Example, display the result (True/False) of each test.

1. dsep(D;J)
2. dsep(D;J|I,L)
3. dsep(D;J|L)
4. dsep(D;J|I,L,J)
5. dsep(D;I)
6. dsep(D;I|L)
7. dsep(D;S)
8. dsep(D;S|H)
9. dsep(G;S)
10. dsep(G;S|H)

In [ ]:
# **YOUR SOLUTION HERE**

# Use of AI Tools

By submitting this notebook for grading you testify that:

* AI did not draft an earlier version of your work.
* You did not use AI-powered code completion.
* You did not implement algorithms suggested by an AI tool.
* AI did not revise a version of your work.
* You did not implement suggestions made by an AI tool.


_You_ in the sentences above refers to you and all your team members.
_AI_ refers to LM-based tools and assistants (e.g., ChatGPT, Gemini, UvA AI chat, etc.).

If you did make use of an AI tool, you should describe the uses you made of it below. Or indicate that no such tool was used.

**TYPE YOUR STATEMENT HERE**